# Example of Field-aware Factorization Machine with trs.Trainer

In [1]:
from functools import partial
import numpy as np
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.utils.data
import torchtext
import torecsys as trs
from typing import Dict, Tuple, List

c:\program files\python37\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning:

In [2]:
# get samples data from movielens as a example
# trs.data.sampledata.download_ml_data(size="latest-small")
_, movies_df, ratings_df, _ = trs.data.sampledata.load_ml_data(size="latest-small")
# movies_df["year"] = movies_df.title.apply(lambda x: re.findall(r"\((\d+)\)", x))
# movies_df["year"] = movies_df.year.apply(lambda x: int(x[0]) if len(x) > 0 else np.nan)
# movies_df = pd.concat([
#     movies_df, 
#     pd.get_dummies(movies_df.genres.apply(
#         lambda x: x.split("|")).apply(pd.Series).stack()).sum(level=0)
# ], axis=1).drop(["title", "genres"], axis=1)
# merged = pd.merge(ratings_df, movies_df, on="movieId")

In [3]:
user_size = ratings_df.userId.max() + 1
item_size = ratings_df.movieId.max() + 1

embed_size = 16
num_fields = 2

In [4]:
dataset = trs.data.dataset.DataFrameToDataset(ratings_df, columns=["userId", "movieId", "rating"])
schema = {
    "userId": ["user_id", "single_index"],
    "movieId": ["movie_id", "single_index"],
    "rating": ["labels", "values"]
}
collate_fn = partial(trs.data.dataloader.dict_collate_fn, schema=schema)

In [5]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [6]:
# inititalize embedding fields
feat_inputs_embedding = trs.inputs.base.MultipleIndexEmbedding(
    1, [user_size, item_size]
)
field_aware_embedding = trs.inputs.base.FieldAwareMultipleIndexEmbedding(
    embed_size, [user_size, item_size]
)

# define schema of wrapper and initialize InputsWrapper
schema = {
    "feat_inputs"      : (feat_inputs_embedding, ["user_id", "movie_id"]),
    "field_emb_inputs" : (field_aware_embedding, ["user_id", "movie_id"])
}
inputs_wrapper = trs.inputs.InputsWrapper(schema)

C:\Users\User\Desktop\torecsys\torecsys\utils\decorator\decorator.py:49: UserWarning: the module have been checked with torch.jit.trace, but the feature is in experimemtal.!!! Remark. It is not allow to use dropout at this stage, since inferences of the same inputs will be different after dropout layer. So, Please use dropout_p = 0.0 or None.
  warnings.warn(msg)


In [7]:
# initialize field-aware factorizatiob machine model
ffm = trs.models.FieldAwareFactorizationMachineModel(embed_size, num_fields)

In [8]:
trainer = trs.Trainer(
    inputs_wrapper = inputs_wrapper, 
    model = ffm
)

logger have been initialized.
tensorboard summary writter have been created and the log directory is set to C:\Users\User\Desktop\torecsys\torecsys\utils\training\logdir.
+------------------------------------------------------------------------------+
|     Name:                                  Value:                            |
| Embeddings      InputsWrapper                                                |
| Model           FieldAwareFactorizationMachineModel                          |
| Loss            MSELoss                                                      |
| Optimizer       AdamW                                                        |
| Reg norm        2                                                            |
| Reg lambda      0.100                                                        |
| Num of epochs   10                                                           |
| Log directory   C:\Users\User\Desktop\torecsys\torecsys\utils\training\logdi |
|                 r

In [ ]:
trainer.fit(dataloader)

Epoch 1 / 10:


step avg loss : 0.1184


# Inputs

In [ ]:
emb_dict = torch.jit.trace(trs.inputs.base.EmbeddingDict(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), {"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})
emb_dict({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

In [ ]:
fa_emb = torch.jit.trace(trs.inputs.base.FieldAwareMultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
fa_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
list_emb = torch.jit.trace(trs.inputs.base.ListIndexEmbedding(16, 100, num_heads=4), torch.Tensor([[1, 4, 5]]).long())
list_emb(torch.Tensor([[1, 4, 5]]).long())
# try to fix if use trace
# list_emb.show_attention(torch.Tensor([[1, 4, 5]]).long())

In [ ]:
mlp_emb = torch.jit.trace(trs.inputs.base.MultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
mlp_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
sq_emb = torch.jit.trace(trs.inputs.base.SequenceIndexEmbedding(16, 100), (torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long()))
sq_emb(torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long())

In [ ]:
index_emb = torch.jit.trace(trs.inputs.base.SingleIndexEmbedding(16, 100), torch.Tensor([[1]]).long())
index_emb(torch.Tensor([[1]]).long())

In [ ]:
stack = torch.jit.trace(trs.inputs.base.StackedInputs(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), ({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())}))
stack({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

In [ ]:
val = torch.jit.trace(trs.inputs.base.ValueInputs(5), torch.randn(size=(1, 5)))
val(torch.randn(size=(1, 5)))

In [ ]:
img_inputs = torch.jit.trace(trs.inputs.base.ImageInputs(16, 3, [8, 8], [1, 1], [1, 1], [1, 1]), torch.randn(4, 3, 64, 64))
img_inputs(torch.randn(4, 3, 64, 64))

In [ ]:
pre_img = torch.jit.trace(trs.inputs.base.PretrainedImageInputs(16, "resnet18"), torch.randn(4, 3, 64, 64))
pre_img(torch.randn(4, 3, 64, 64))

In [ ]:
inputs = {
    "sequenceId"     : torch.randint(100, size=(8, 4)),
    "sequenceLength" : torch.Tensor([4] * 8),
    "values"         : torch.randn(size=(8, 8)),
    "userId"         : torch.randint(100, size=(8, 1))
}

In [ ]:
cat_schema = [
    (trs.inputs.base.SequenceIndexEmbedding(8, 100), ["sequenceId"], ["sequenceLength"]),
    (trs.inputs.base.ValueInputs(8), ["values"])
]

In [ ]:
stacked_schema = [
    (trs.inputs.base.SingleIndexEmbedding(16, 100), ["userId"]), 
    (trs.inputs.base.ConcatInputs(cat_schema), ["sequenceId", "sequenceLength", "values"])
]

In [ ]:
stacked = trs.inputs.base.StackedInputs(stacked_schema)

In [ ]:
wrapped_schema = {
    "stacked": (trs.inputs.base.StackedInputs(stacked_schema), ["userId", "sequenceId", "sequenceLength", "values"])
}

In [ ]:
wrapped = trs.inputs.InputsWrapper(wrapped_schema)

# Models

In [ ]:
e = 8
n = 5
b = 16

feat_inputs = torch.randn(size=(b, n, 1))
field_emb = torch.randn(size=(b, n, e))
field_aware_emb = torch.randn(size=(b, n * n, e))

In [ ]:
user_item_emb = torch.randn(size=(b, 2, e))

In [ ]:
afm_model = torch.jit.trace(trs.models.AttentionalFactorizationMachineModel(e, n, 8, 0.0), (feat_inputs, field_emb))
afm_model(feat_inputs, field_emb)

In [ ]:
dc_model = torch.jit.trace(trs.models.DeepAndCrossNetworkModel(e * n, 1, [10], 2, 1, [0.0], nn.ReLU6()), field_emb)
dc_model(field_emb)

In [ ]:
dffm_model = torch.jit.trace(trs.models.DeepFieldAwareFactorizationMachineModel(e, n, 1, [10], 0.0, [0.0], nn.Tanh(), 1), field_aware_emb)
dffm_model(field_aware_emb)

In [ ]:
dfm_model = torch.jit.trace(trs.models.DeepFactorizationMachineModel(e, n, [10], 0.0, [0.0], nn.ReLU(), 1), (feat_inputs, field_emb))
dfm_model(feat_inputs, field_emb)

In [ ]:
fm = torch.jit.trace(trs.models.FactorizationMachineModel(e, n, 0.1), (feat_inputs, field_emb))
fm(feat_inputs, field_emb)

In [ ]:
fmnn = torch.jit.trace(trs.models.FactorizationMachineSupportedNeuralNetwork(e, n, 1, [10], 0.1, [0.1], nn.Tanh()), (feat_inputs, field_emb))
fmnn(feat_inputs, field_emb)

In [ ]:
fatdffm = torch.jit.trace(trs.models.FieldAttentiveDeepFieldAwareFactorizationMachineModel(e, n, 10, [10], 16, 0.0, [0.1], nn.Tanh()), field_aware_emb)
fatdffm(field_aware_emb)

In [ ]:
ffm = torch.jit.trace(trs.models.FieldAwareFactorizationMachineModel(e, n), (feat_inputs, field_aware_emb))
ffm(feat_inputs, field_aware_emb)

In [ ]:
lr = torch.jit.trace(trs.models.LogisticRegressionModel(n), feat_inputs)
lr(feat_inputs)

In [ ]:
ncf = torch.jit.trace(trs.models.NeuralCollaborativeFilteringModel(e, 1, [10], [0.1], nn.ReLU6()), user_item_emb)
ncf(user_item_emb)

# Layers

In [ ]:
afm = trs.layers.AttentionalFactorizationMachineLayer(e, n, attn_size=8, dropout_p=0.0)
print(afm(field_emb)[0].size())
print(afm(field_emb)[1].size())

In [ ]:
blin = trs.layers.BilinearNetworkLayer(e, n, 1, 3)
blin(field_emb).size()

In [ ]:
cen = trs.layers.ComposeExcitationNetworkLayer(n)
cen(field_aware_emb).size()

In [ ]:
cin = trs.layers.CompressInteractionNetworkLayer(e, n, 1, [10, 10, 10])
cin(field_emb).size()

In [ ]:
cross = trs.layers.CrossNetworkLayer(5, e, n)
cross(field_emb).size()

In [ ]:
fm = trs.layers.FactorizationMachineLayer(0.0)
fm(field_emb).size()

In [ ]:
ffm = trs.layers.FieldAwareFactorizationMachineLayer(n, 0.0)
ffm(field_aware_emb).size()

In [ ]:
inner = trs.layers.InnerProductNetworkLayer(n)
inner(field_emb).size()

In [ ]:
mlp = trs.layers.MultilayerPerceptronLayer(1, [10, 10, 10], embed_size=e, num_fields=n, dropout_p=[0.0, 0.0, 0.0])
mlp(field_emb).size()

In [ ]:
outer = trs.layers.OuterProductNetworkLayer(e, n)
outer(field_emb).size()

In [ ]:
wide = trs.layers.WideLayer(e, n, 1)
wide(field_emb).size()